In [1]:
import csv
import cv2
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [10]:
train_data=[]
target=[]
with open(r'C:\Users\Student\Desktop\dataset_1\train.csv') as dat:
    dat.readline()
    reader = csv.reader(dat, delimiter = ',')
    for row in reader:
        temp=cv2.imread(os.path.join(os.getcwd(), 'train', row[0]))
       # temp1=cv2(temp)
        train_data.append(temp)
        target.append(row[1])
        


In [3]:
  print(type(train_data))
    
print(train_data[1])

<class 'list'>
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [4]:
train_data=np.array(train_data)
train_data=train_data/255.
target=np.array(target)

uni=np.unique(target)
print(uni)

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']


In [5]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = target.reshape(len(target), 1)
Y = onehot_encoder.fit_transform(integer_encoded)
print(Y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:

test_data=[]

with open('test_WyRytb0.csv') as dat:
    dat.readline()
    reader = csv.reader(dat, delimiter = ',')
    for row in reader:
        temp=cv2.imread(os.path.join(os.getcwd(), 'train', row[0]))
        temp1=cv2.resize(temp, (100,100))
        test_data.append(temp1)
        
        
test_data=np.array(test_data)
print(test_data.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'test_WyRytb0.csv'

In [7]:
img=cv2.imread(r"C:\\Users\\Student\\Desktop\\dataset_1\\train\\0.png")
#imgplot = plt.imshow(img)
dimensions = img.shape
print(dimensions)

(28, 28, 3)


In [45]:
def IntelNet(input_shape = (28, 28, 3), classes = len(uni)):
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    
     # Stage 1
    X = Conv2D(128, (7, 7), strides = (2, 2), name = 'conv1',padding = 'SAME')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    #X = MaxPooling2D((3, 3), strides=(2, 2),padding = 'SAME')(X)
    

    X = Conv2D(128, (5, 5), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    #X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    X = Conv2D(128, (5, 5), strides = (1, 1), name = 'conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3, 3), strides = (1, 1), name = 'conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv4')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    
    
    
    
    
    
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [51]:
model = IntelNet(input_shape = (28, 28, 3), classes = len(uni))

In [52]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [53]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(train_data,Y,test_size = 0.2)

In [54]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 100)

Epoch 1/100
4800/4800 [==============================] - 96s 20ms/step - loss: 0.6681 - acc: 0.8167
Epoch 2/100
4800/4800 [==============================] - 90s 19ms/step - loss: 0.1407 - acc: 0.9560
Epoch 3/100
4800/4800 [==============================] - 91s 19ms/step - loss: 0.0909 - acc: 0.9696
Epoch 4/100
4800/4800 [==============================] - 90s 19ms/step - loss: 0.0513 - acc: 0.9854
Epoch 5/100
4800/4800 [==============================] - 90s 19ms/step - loss: 0.0397 - acc: 0.9900
Epoch 6/100
4800/4800 [==============================] - 90s 19ms/step - loss: 0.0223 - acc: 0.9956
Epoch 7/100
4800/4800 [==============================] - 90s 19ms/step - loss: 0.0100 - acc: 0.9992
Epoch 8/100
4800/4800 [==============================] - 90s 19ms/step - loss: 0.0069 - acc: 0.9996
Epoch 9/100
4800/4800 [==============================] - 90s 19ms/step - loss: 0.0039 - acc: 1.0000
Epoch 10/100
4800/4800 [==============================] - 90s 19ms/step - loss: 0.0021 - acc: 1.0000

In [57]:
model.evaluate(x=X_test, y=Y_test, batch_size=10)

1200/1200 [==============================] - 5s 4ms/step


[0.0762311013258755, 0.9799999957283337]

In [58]:
probs=model.predict(X_test)
classe=probs.argmax(axis=-1)
print(classe)

[7 4 0 ... 2 9 0]


In [59]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 3)         0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 34, 34, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 17, 17, 128)       18944     
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 17, 17, 128)       512       
_________________________________________________________________
activation_5 (Activation)    (None, 17, 17, 128)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 13, 13, 128)       409728    
_________________________________________________________________
bn_conv2 (BatchNormalization (None, 13, 13, 128)       512       
__________